In [93]:
import requests
import pandas as pd
import re
import operator
import time
import json
import urllib.request
import os
import html as ihtml


In [4]:
#Function credit to https://github.com/abigailhaddad/USAJobsPersonnelData
'''   
def connect(authorization_key):
    #passes key to the API
    headers = {'Authorization-Key': authorization_key,
 'Host': 'data.usajobs.gov',
 'User-Agent': 'jware5@gmu.edu'}
    return headers

def format_dict(results, df_fixed):
    #formats the results of current_search into a df
    for j in range(0, len(results['SearchResult']['SearchResultItems'])):
        df_fixed=df_fixed.append(pd.DataFrame.from_dict(results['SearchResult']['SearchResultItems'][j]['MatchedObjectDescriptor'], orient='index').transpose())
        df_fixed=df_fixed.apply(pd.Series)
    return(df_fixed)

def current_search(authorization_key, JobCategoryCode="", terms=""):
    #formats url and makes request to API
    number=str(1)
    base_url=f"https://data.usajobs.gov/api/search?&JobCategoryCode={JobCategoryCode}&Keyword={terms}&WhoMayApply=All&p="
    df_fixed=pd.DataFrame()
    results = requests.get(base_url+number, headers=connect(authorization_key)).json()
    df_fixed=format_dict(results, df_fixed)
    if len(results['SearchResult']['SearchResultItems']) == 25:
            while results['SearchResult']['SearchResultCount']!= 0 and int(number)<101:
                number=str(int(number)+1)      
                results = requests.get(base_url+number, headers=connect(authorization_key)).json()
                df_fixed=format_dict(results, df_fixed)
    #dfArmy=df_fixed.loc[df_fixed['DepartmentName']=="Department of the Army"]
    #return(dfArmy)
    return(df_fixed)
'''


In [7]:
#Testing USAJobs API
'''
authkey = 'iQhjQKEYw9gxaHYH8SdlujF8Gxt8Gg51P8T4desTHpY='
number = str(1)
def connect(authorization_key):
    #passes key to the API
    headers = {'Authorization-Key': authorization_key,
 'Host': 'data.usajobs.gov',
 'User-Agent': 'jware5@gmu.edu'}
    return headers

#Modified number of results per page to 500;
base_url=f"https://data.usajobs.gov/api/search?&ResultsPerPage=500&JobCategoryCode=""&Keyword=cybersecurity&WhoMayApply=All&p="
results = requests.get(base_url+number, headers=connect(authkey)).json()

#This is likely the target element for alignment
results['SearchResult']['SearchResultItems'][1]['MatchedObjectDescriptor']['UserArea']['Details']['MajorDuties']
'''

In [75]:
#Process is slightly different for each calander year of USAJobs data; the CSV is used to identify job titles
#consistent with cybersecurity jobs: IT Specialist, Information Security, Cybersecurity Specialist, etc.

#Unfortunately, the CSV structure varies from year to year, including some encoding errors that must be addressed.

#CY20 is a comma delimited type; it has an extra column
df_cy20 = pd.read_csv('D:\\capstone\\CY20\\FP9434.csv', header = 0, names = ['ControlNum', 'StartDtd','Expire', 'EndDtd', 'PositionTitle'], encoding = "ISO-8859-1")
#CY19 is a tab delimited type
df_cy19 = pd.read_csv('D:\\capstone\\CY19\\FP8794.csv', sep='\t', header = 0,names = ['ControlNum', 'StartDtd', 'EndDtd', 'PositionTitle'], encoding = "ISO-8859-1")
#CY18 is a comma delimited type
df_cy18 = pd.read_csv('D:\\capstone\\CY18\\CY18\\CY18.csv', header = 0,names = ['ControlNum', 'StartDtd', 'EndDtd', 'PositionTitle'], encoding = "ISO-8859-1")
#CY17 is a comma delimited type
df_cy17 = pd.read_csv('D:\\capstone\\CY17\\CY17\\CY17.csv', header = 0,names = ['ControlNum', 'StartDtd', 'EndDtd', 'PositionTitle'], encoding = "ISO-8859-1")

In [76]:
#Combine into single dataframe
pdList = [df_cy20.drop(columns=['Expire']), df_cy19, df_cy18, df_cy17]
new_df = pd.concat(pdList, ignore_index = True)
new_df

,ControlNum,StartDtd,EndDtd,PositionTitle
0,465821900,3/16/2017,3/20/2017,STUDENT TRAINEE
1,468024400,4/18/2017,5/2/2017,Student Trainee (Repayment)
2,469059100,5/3/2017,5/24/2017,Electrician
3,469060600,5/3/2017,5/24/2017,Electrician
4,469676500,5/11/2017,6/1/2017,SUPERVISORY PROGRAM ANALYST
...,...,...,...,...
1299793,487727200,12/31/2017,1/12/2018,Ordering Technician
1299794,487727300,12/31/2017,1/12/2018,Ordering Technician
1299795,487727400,12/31/2017,1/12/2018,Ordering Technician (Recent Graduate)
1299796,487727500,12/31/2017,1/8/2018,Human Resources Specialist (Recruitment and Pl...


In [98]:
#Extract positions containing Cyber and INFOSEC strings
df = new_df[new_df['PositionTitle'].str.contains("Cyber|INFOSEC|Information Security",case = False)]
df = df.reset_index(drop = True)

In [86]:
#Move filtered xml files to new location; customized for CY20 and CY19
import shutil
paths = ['D:\\capstone\\CY19\\','D:\\capstone\\CY20\\','D:\capstone\\CY18\CY18\\','D:\capstone\\CY17\CY17\\']
dst_path = 'D:\capstone\\Federal_CyberJobs\\'


In [159]:
#This moves all xml files that match the cyber security regex filter.
for path in paths:
    for filename in os.listdir(path):
        try:
            if int(filename[0:9]) in df.ControlNum.values:
                shutil.move(path + filename, dst_path)
        except:
            continue

In [94]:
import bs4
import lxml
from bs4 import BeautifulSoup

In [95]:
#<JobCategoryCode>2210</JobCategoryCode>

#<JobGradeCode>GS</JobGradeCode>

#<PositionOfferingTypeCode>15317</PositionOfferingTypeCode>
#<GOVT_PayGradeLow>13</GOVT_PayGradeLow>

#<GOVT_PayGradeHigh>13</GOVT_PayGradeHigh>
#<RemunerationMinimumAmount>90823</RemunerationMinimumAmount>

#<RemunerationMaximumAmount>118069</RemunerationMaximumAmount>
def clean_text(text):
    text = BeautifulSoup(ihtml.unescape(text), "lxml").text
    text = re.sub(r"\s+", " ", text)
    return text

#Create blank dataframe to populate
jobs_df = pd.DataFrame(columns = ["Title", "GS_Low", "GS_High", "Department", "Organization", "Duties", "Qualifications"])



In [99]:
#Parse through xml files
path = "D:\\capstone\\USAJobs Data\\"
for filename in os.listdir(path):
    with open(f"{path}{filename}", 'r') as f:
        try:
            
            file = f.read()
            soup = BeautifulSoup(file, 'lxml')
            title = soup.find('positiontitle').get_text()
            gs_low = soup.find('govt_paygradelow').get_text()
            gs_high = soup.find('govt_paygradehigh').get_text()
            department = soup.find('display_department').get_text()
            org = soup.find('organizationname').get_text()
            duties = clean_text(soup.find('duties').get_text())
            quals = clean_text(soup.find('qualifications').get_text())
            pd.DataFrame([[title, gs_low, gs_high, department, org, duties, quals]]).to_csv('USAJobs.csv', mode='a', index=False, header=False)
        except:
            #skip problem files
            continue